https://towardsdatascience.com/simple-ocr-with-tesseract-a4341e4564b6

In [1]:
import os
from PIL import Image
from matplotlib import pyplot as plt
import pytesseract
import numpy as np
import cv2

In [2]:
font = list(range(0x7f))
for a in range(0x20, 0x7f):
    font[a] = Image.open("font/" + hex(a)[2:] + ".png")

In [3]:
def gen_image(text):
    pad = 4
    width = 0
    for a in text:
        width += font[ord(a)].size[0] + 1
    width += pad*2
    height = font[ord('A')].size[1] + pad*2
    img = Image.new(mode = "RGB", size = (width, height), color = (255, 255, 255))
    
    boxes = []
    left = pad
    for a in text:
        letter = font[ord(a)]
        top = height - letter.height - pad
        img.paste(letter, (left, pad))
        boxes.append((a, left, top, left + letter.width, height - pad))
        left += letter.size[0] + 1
    return (img, boxes)

In [4]:
text = ''
for a in range(0x21, 0x7f):
    text = text + chr(a)
text

'!"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

In [5]:
gen, boxes = gen_image(text)
gen

In [6]:
%time pytesseract.image_to_string(gen)

Wall time: 154 ms


'VWH$BS’C*+,-./0123456789:}:\n\n-? @ABCDEFGHIJIKLMNOPQRSTUYWXY ZEN 1*_‘abedef ghijkimnopgrstuvwxyz{l}”\n'

In [7]:
boxes_lines = []
for params in boxes:
    boxes_lines.append('%s %d %d %d %d %d' % (*params, 0))

Generate other size images

In [8]:
os.makedirs("train", exist_ok=True)

In [9]:
pages = []
page_ind = 0

def add_page(sf, mode):
    global page_ind
    page_ind += 1
    im = gen.resize((gen.width*sf, gen.height*sf), mode)
    pages.append(im)
    for params in boxes:
        (c,l,t,r,b) = params
        boxes_lines.append('%s %d %d %d %d %d' % (c, l*sf, t*sf, r*sf, b*sf, page_ind))

In [10]:
add_page(2, Image.ANTIALIAS)
add_page(2, Image.NEAREST)
add_page(3, Image.ANTIALIAS)
add_page(3, Image.NEAREST)
add_page(4, Image.ANTIALIAS)
add_page(4, Image.NEAREST)

In [11]:
with open('train/lb.font.exp0.box', 'w') as f:
    for line in boxes_lines:
        f.write(line)
        f.write('\n')

In [12]:
# Merge tif
gen.save("train/lb.font.exp0.tif", format="tiff", append_images=pages, save_all=True)

In [22]:
with open('train/font_properties', 'w') as f:
    f.write("font 0 0 0 0 0")

In [23]:
%cd train

C:\Projects\Translate\video_text\work\train


In [24]:
# Create a .tr file (training file)
!tesseract lb.font.exp0.tif lb.font.exp0 nobatch box.train

Page 1
Estimating resolution as 111
FAIL!
APPLY_BOXES: boxfile line 2/" ((7,10),(12,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 12/, ((77,3),(80,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 13/- ((81,4),(87,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 14/. ((88,3),(90,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 26/: ((171,5),(173,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 28/< ((178,3),(184,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 29/= ((185,6),(193,12)): FAILURE! Couldn't find a matching blob
FAIL!
APPLY_BOXES: boxfile line 30/> ((194,3),(200,12)): FAILURE! Couldn't find a matching blob
APPLY_BOXES:
   Boxes read from boxfile:      94
   Boxes failed resegmentation:       8
APPLY_BOXES: Unlabelled word at :Bounding box=(178,3)->(183,12)
APPLY_BOXES: Unlabelled word at :Bounding box=(194,3

In [25]:
# Create a unicharset file
!unicharset_extractor lb.font.exp0.box

Extracting unicharset from box file lb.font.exp0.box
Wrote unicharset file unicharset


In [29]:
# Create a shapetable file
!shapeclustering -F font_properties -U unicharset -O lb.unicharset lb.font.exp0.tr

In [39]:
# Create a pffmtable, intemp file
!mftraining -F font_properties -U unicharset -O lb.unicharset lb.font.exp0.trecho

Reading lb.font.exp0.trecho ...
Failed to open tr file: lb.font.exp0.trecho
Flat shape table summary: Number of shapes = 0 max unichars = 0 number with multiple unichars = 0
This application has requested the Runtime to terminate it in an unusual way.
Please contact the application's support team for more information.
reateIntTemplates()
Class->NumConfigs == this->fontset_table_.at(Class->font_set_id).size():Error:Assert failed:in file ../../../src/classify/intproto.cpp, line 948


In [41]:
# Create a normproto file
!cntraining lb.font.exp0.tr

In [21]:
%cd -

C:\Projects\Translate\video_text\work
